# Data Science Immersive - General Assembly 
## Capstone Project
Harsha Goonewardana <br>
Today

## Natural Language Processing
### This notebook will segregate the top 25% and the bottom 25% in price and conduct unseurvised learning via a latent Dirichlet allocation model

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None
import seaborn as sns
import textacy
import pickle

In [7]:
import nltk

from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel

import itertools
from collections import Counter
from collections import defaultdict

import json
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [8]:
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn

In [9]:
import string

In [10]:
df= pd.read_pickle('./data/airbnbNYCclean.pkl')

In [11]:
#the size of each description field was 7 million words which is too big for textcy/ spacy  to handle 

In [12]:
df.head(5)

,description,number_of_reviews,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,security_deposit,amenities,house_rules,host_since,host_is_superhost,cancellation_policy,neighbourhood_cleansed,reviews_per_month,latitude,longitude,bed_type,ppl_bath,guest_prop,bnb_yield,tier,tier_price,DateDiffHostSince,Location,dist_transit
0,Step into our artistic spacious apartment and ...,168.0,Apartment,Private room,3.0,1.0,1.0,2.0,119.0,2.0,39.0,2.0,0.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",no-smoking/please take off your shoes: cleanin...,2008-09-06,f,strict_14_with_grace_period,Harlem,1.43,40.799205,-73.953676,Real Bed,0.5,0.666667,17425.408,2nd,3rd,3601.0,"(40.79920479936168, -73.95367574543542)",0.115468
1,Renovated apt home in elevator building. Spaci...,7.0,Apartment,Private room,4.0,1.0,2.0,3.0,104.0,1.0,25.0,1.0,100.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",-The security and comfort of all our guests is...,2008-09-07,t,moderate,Kensington,0.23,40.647486,-73.972370,Real Bed,1.0,0.250000,2449.408,3rd,3rd,3600.0,"(40.64748608166989, -73.97236954007957)",0.392200
2,"Find your romantic getaway to this beautiful, ...",32.0,Apartment,Entire home/apt,2.0,1.0,0.0,1.0,325.0,2.0,0.0,1.0,350.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...","Make yourself at home, respect the space and t...",2008-09-09,f,strict_14_with_grace_period,Midtown,0.31,40.753621,-73.983774,Real Bed,0.5,1.000000,10316.800,3rd,1st,3598.0,"(40.753620726572464, -73.98377381114605)",0.085077
3,"This is a spacious, clean, furnished master be...",32.0,Apartment,Private room,2.0,1.0,1.0,1.0,195.0,2.0,50.0,5.0,150.0,"{TV,Internet,Wifi,""Wheelchair accessible"",Kitc...",Guests should be respectful of the home and th...,2008-11-09,f,strict_14_with_grace_period,Williamsburg,0.31,40.708558,-73.942362,Real Bed,0.5,1.000000,6190.080,3rd,2nd,3537.0,"(40.70855778333159, -73.94236227640226)",0.216335
4,Urban retreat: enjoy 500 s.f. floor in 1899 br...,198.0,Other,Entire home/apt,3.0,1.0,1.0,3.0,89.0,1.0,0.0,1.0,500.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Smoking - outside please; pets allowed but ple...,2008-12-07,f,moderate,Clinton Hill,4.42,40.685138,-73.959757,Real Bed,1.0,0.333333,40282.112,2nd,3rd,3509.0,"(40.68513770564301, -73.95975749881094)",0.403653


In [13]:
df=df[['description','tier']]

In [14]:
def process_text(txt):
    '''
    Return cleansed text by tokenizing, stripping stopwords, and lemmatizing the tokens. This function is used prior to performing text analysis and topic modeling.
    '''
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(str(txt).lower())

    sw = set(stopwords.words('english'))
    stopped_tokens = [i for i in tokens if not i in sw]

    wordnet = WordNetLemmatizer()
    clean_txt = ' '.join([wordnet.lemmatize(i) for i in stopped_tokens])

    return clean_txt


### Top 25 Percent

In [16]:
# preprocess the observations for topic modelling
X = df[df['tier'] == '1st']['description'].values
new_X = []
for description in X:
    new_X.append(process_text(description))

max_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=max_features, stop_words='english')

tf = tf_vectorizer.fit_transform(new_X)

In [17]:
n_topics = 4
lda_model = LatentDirichletAllocation(n_topics=n_topics, max_iter=15, learning_method='online', learning_offset=10., n_jobs=-1, random_state=42)

lda_model.fit(tf)
vis_data = pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer, R=10, n_jobs=-1)
# pyLDAvis.display(vis_data)
pyLDAvis.show(vis_data)
#     pyLDAvis.save_html(vis_data, 'web_app/templates/pylda.html')

/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8891/    [Ctrl-C to exit]


127.0.0.1 - - [17/Jul/2018 10:07:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 10:08:00] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 10:08:00] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 10:08:00] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 10:08:01] code 404, message Not Found
127.0.0.1 - - [17/Jul/2018 10:08:01] "GET /favicon.ico HTTP/1.1" 404 -



stopping Server...


### Topic 4 seems to be 

In [18]:
# Attrib https://stackoverflow.com/questions/44208501/getting-topic-word-distribution-from-lda-in-scikit-learn
vocab = tf_vectorizer.get_feature_names()
topic_words = {}
n_top_words = 10
for topic, comp in enumerate(lda_model.components_):
    # for the n-dimensional array "arr":
    # argsort() returns a ranked n-dimensional array of arr, call it "ranked_array"
    # which contains the indices that would sort arr in a descending fashion
    # for the ith element in ranked_array, ranked_array[i] represents the index of the
    # element in arr that should be at the ith index in ranked_array
    # ex. arr = [3,7,1,0,3,6]
    # np.argsort(arr) -> [3, 2, 0, 4, 5, 1]
    # word_idx contains the indices in "topic" of the top num_top_words most relevant
    # to a given topic ... it is sorted ascending to begin with and then reversed (desc. now)    
    word_idx = np.argsort(comp)[::-1][:n_top_words]

    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]

In [19]:
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))


Topic: 0
  bedroom, apartment, room, floor, bed, private, living, kitchen, bathroom, space
Topic: 1
  apartment, new, city, york, east, nyc, village, located, space, bedroom
Topic: 2
  bed, apartment, bedroom, kitchen, room, queen, tv, size, new, living
Topic: 3
  apartment, minute, block, park, subway, walk, away, restaurant, train, time


### Bottom 25 percent 

In [ ]:

X = df[df['tier_price'] == '4th']['description'].values
new_X = []
for description in X:
    new_X.append(process_text(description))

max_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=max_features, stop_words='english')

tf = tf_vectorizer.fit_transform(new_X)

In [ ]:
n_topics = 4
lda_model = LatentDirichletAllocation(n_topics=n_topics, max_iter=15, learning_method='online', learning_offset=10., n_jobs=-1, random_state=42)

lda_model.fit(tf)
vis_data = pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer, R=10, n_jobs=-1)
pyLDAvis.display(vis_data)
#     pyLDAvis.show(vis_data)


In [ ]:
#attribution https://github.com/Stuart-D-King/the_hub
vocab = tf_vectorizer.get_feature_names()
topic_words = {}
n_top_words = 10
for topic, comp in enumerate(lda_model.components_):
    # for the n-dimensional array "arr":
    # argsort() returns a ranked n-dimensional array of arr, call it "ranked_array"
    # which contains the indices that would sort arr in a descending fashion
    # for the ith element in ranked_array, ranked_array[i] represents the index of the
    # element in arr that should be at the ith index in ranked_array
    # ex. arr = [3,7,1,0,3,6]
    # np.argsort(arr) -> [3, 2, 0, 4, 5, 1]
    # word_idx contains the indices in "topic" of the top num_top_words most relevant
    # to a given topic ... it is sorted ascending to begin with and then reversed (desc. now)    
    word_idx = np.argsort(comp)[::-1][:n_top_words]

    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]

In [ ]:
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))



### Hosts clusters 